In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Reading in and preprocessing data

In [ ]:
# Reading in data from the 50-50 readmit vs. nonreadmit dataset

df = pd.read_csv('dfd.csv')
df.head()

In [ ]:
# Dropping certain columns

# Dropping ID numbers and dates
df = df.drop(columns=['subject_id', 'hadm_id', 'admittime', 'dischtime'])

# Dropping labevents and chartevents values of less importance based on feature selection (2/3 values for each measurement)
df = df.drop(columns=['rdw_min', 'rdw_max', 'hemoglobin_min', 'hemoglobin_max', 'creatinine_median', 'creatinine_min', 
                      'hematocrit_median', 'hematocrit_min', 'tempc_median', 'tempc_max', 'resprate_median', 
                      'resprate_min', 'wbc_median', 'wbc_max', 'inr_min', 'inr_median', 'ptt_median', 'ptt_max', 
                      'lactate_median', 'lactate_max', 'sysbp_median', 'sysbp_min', 'spo2_median', 'spo2_max', 
                      'bilirubin_median', 'bilirubin_max', 'platelet_median', 'platelet_max', 'heartrate_min',
                      'heartrate_median'])
df.head()

In [ ]:
# Converting categorical features into dummy variables

df_converted = pd.get_dummies(df)
df_converted.head()

In [ ]:
# Splitting dataframe into data (predictors) vs. label (attributed to be predicted)

label_df = df_converted.pop('followed_by_readmit')
data_df = df_converted
print('label_df:\n', label_df.head(), 2*'\n', 'data_df:\n', data_df.head())

In [ ]:
# Converting dataframes to NumPy arrays

label = label_df.values
data = data_df.values

In [ ]:
label

In [ ]:
data

## Train/test split

In [ ]:
# 80/20 train-test split

from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(data, label, train_size=0.8, test_size=0.2, random_state=10)

print('Training data:', train_data.shape, '\tTest data:', test_data.shape)
print('Training labels:', train_label.shape, '\tTest labels:', test_label.shape)

In [ ]:
# Cross-validation on the training set (no need to do explicitly for Logistic Reg. since has a cv parameter, but just in case...)

#from sklearn.model_selection import KFold
#kf = KFold(n_splits=5, random_state=10)
#for train, test in kf.split(train_data, train_label):
    #print(test)

Note that cross-validation will be performed with the training data for each machine learning algorithm tested below.

## Logistic regression

From http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression:
"LogisticRegressionCV implements Logistic Regression with builtin cross-validation to find out the optimal C parameter (similar to what GridSearchCV might do)."

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

First, let's try using the `liblinear` solver (recommended for small datasets) using 'l1' penalty:

In [ ]:
# Instantiating logistic regression estimator object; cv=5 indicates 5-fold cross-validation

lrcv_l1 = LogisticRegressionCV(cv=5, penalty='l1', solver='liblinear', random_state=10)

In [ ]:
%%time

lrcv_l1.fit(train_data, train_label)

In [ ]:
lrcv_l1_pred = lrcv_l1.predict(test_data)
lrcv_l1_pp = lrcv_l1.predict_proba(test_data)

Running self.classes_ on the result from fitting the model to the data (see commented code below) tells us that the probability estimates for positive class ("True") are in column 1 rather than column 0 of the result from `predict_proba()` - this is what we'll need to use when calculating the ROC-AUC.

In [ ]:
#lrcv_l1.fit(train_data, train_label).classes_

In [ ]:
from sklearn.metrics import roc_auc_score

lrcv_l1_probs = lrcv_l1.predict_proba(test_data)[:,1]
print('ROC-AUC score with l1 penalty:', roc_auc_score(test_label, lrcv_l1_probs))

What if we used 'l2' penalty instead?

In [ ]:
%%time
# Repeating the code above for logistic regression but with l2 penalty

lrcv_l2 = LogisticRegressionCV(cv=5, penalty='l2', solver='liblinear', random_state=10)
lrcv_l2.fit(train_data, train_label)
lrcv_l2_pred = lrcv_l2.predict(test_data)
lrcv_l2_pp = lrcv_l2.predict_proba(test_data)
lrcv_l2_probs = lrcv_l2.predict_proba(test_data)[:,1]
print('ROC-AUC score with l2 penalty:', roc_auc_score(test_label, lrcv_l2_probs))

Using the liblinear solver, l1 penalty has a very slightly better ROC-AUC score.

Now, let's get the confusion matrix and precision/recall/F1-score.

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_label, lrcv_l1_pred)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(test_label, lrcv_l1_pred))